<hr>
<center><strong>  Taller de Optimización y Simulaciones <br> Simulación Montecarlo <br>Teoría de Colas<br>
<hr>
Taller 1
</strong></center>
<hr>
<center><strong> Hecho por Alejandro Uscátegui </strong></center>
<hr>
<center><strong> Universidad Javeriana <br> 2023 <br> Prof. Gabriel Zambrano  </strong></center>
<hr>

In [2]:
import pandas as pd
import math
import numpy as np
from pulp import *
import xlsxwriter
import random

<i>Indicamos las constantes, creamos la tabla e indicamos las variables</i>

1. Sabemos que se cumple la condición de estabilidad pues lambda es menor a miu


In [7]:
# Constantes

lambdaa = 0.05
miu = 0.08
acumuladordetiempos = 0

# Modelo de Little para el sistema de colas

TiempoPromedioEnSistemaTeorico = 1/(miu - lambdaa)
timepoPromedioEnEspera = lambdaa/(miu*(miu - lambdaa))
tiempoPromedioServicio = 1/miu 
utilizaciondelsistemaTeorico = lambdaa/miu
CantidadPromedioDeOrdenesEnSistemaTeorico = lambdaa/(miu - lambdaa)
CantidadPromedioOrdenesEnServicioTeorico = utilizaciondelsistemaTeorico

# Columna de número de replicas

Nreplicas = 1000

# Pensamos que debíamos hacer por listas el programa, pues así podríamos relacionar facilmente
# las cabeceras de cada columna, en vez de una matriz

clientes = [i+1 for i in range(200)]
# print(clientes)

llegada = [(-1/lambdaa)*math.log(random.random()) for i in range(200)]
# print(llegada)

relojLlegada = []

for i in range(200):
    acumuladordetiempos += llegada[i]
    relojLlegada.append(acumuladordetiempos)
# print(relojLlegada)

tiempodeServicio = [(-1/miu)*math.log(random.random()) for i in range(200)]
# print(tiempodeServicio)

tiempoEnCola = []
tiempoEnCola.append(0)

inicioServicio = []
inicioServicio.append(relojLlegada[0])

findeServicio = []
findeServicio.append(inicioServicio[0]+tiempodeServicio[0])

for i in range(1,200):          
    ### inicioServicio.append(relojLlegada) if relojLlegada[i]>(relojLlegada[i-1] + tiempodeServicio[i -1]) else inicioServicio.append(relojLlegada[i-1] + tiempodeServicio[i -1])
    
    if(relojLlegada[i]>=findeServicio[i-1]):
        tiempoEnCola.append(0)
    else:
        tiempoEnCola.append(findeServicio[i-1]-relojLlegada[i])
    
    if(tiempoEnCola[i] == 0):
        inicioServicio.append(relojLlegada[i])
    else:
        inicioServicio.append(findeServicio[i-1])
    
    findeServicio.append(inicioServicio[i] + tiempodeServicio[i])
        
tiempoTotalSistema = [findeServicio[i] - relojLlegada[i] for i in range(200)]

tiempoReposoSistema = []
tiempoReposoSistema.append(relojLlegada[0])

for i in range(1,200):
    tiempoReposoSistema.append(inicioServicio[i] - findeServicio[i-1])

# Variables simuladas 

TiempoPromedioEnCola_Simulation = sum(tiempoEnCola)/len(tiempoEnCola)
tiempoPromedioEnElSistema_Simulation = sum(tiempoTotalSistema)/len(tiempoTotalSistema)
CantidadPromedioDeOrdenesEnSistema_Simulation = lambdaa*tiempoPromedioEnElSistema_Simulation
CantidadPromedioOrdenesEnCola_Simulation = lambdaa*TiempoPromedioEnCola_Simulation

# Lista que recibe cada variable simulada

tiempos_espera = []
tiempos_espera.append(TiempoPromedioEnCola_Simulation)
tiempos_servicio = []
tiempos_servicio.append(tiempoPromedioEnElSistema_Simulation)
promedios_ordenes_sistema = []
promedios_ordenes_sistema.append(CantidadPromedioDeOrdenesEnSistema_Simulation)
promedio_ordenes_espera = []
promedio_ordenes_espera.append(CantidadPromedioOrdenesEnCola_Simulation)

    
# Línea para crear una matriz de prueba
matriz = np.column_stack((tiempos_espera, tiempos_servicio, promedios_ordenes_sistema, promedio_ordenes_espera))

In [9]:
writer=pd.ExcelWriter('replicas.xlsx', engine='xlsxwriter')
workbook=writer.book

dataf=pd.DataFrame([matriz[i,:] for i in range(1)], columns=['tiempos_espera', 'tiempos_servicio', 'promedios_ordenes_sistema', 'promedio_ordenes_espera'])

dataf.to_excel(
    writer,
    index=False,
    sheet_name= 'prueba'
)

writer.save()

C:\Users\admin\AppData\Local\Temp\ipykernel_14128\2466321811.py:12: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [5]:
""" # Exportamos resultados a excel (sólo para hacer pruebas temporales)

writer=pd.ExcelWriter('pruebas.xlsx', engine='xlsxwriter')
workbook=writer.book

dataf=pd.DataFrame([matriz[i,:] for i in range(200)], columns=['clientes', 'llegada', 'relojLlegada', 'tiempodeServicio', 'tiempoEnCola', 'inicioServicio', 'findeServicio', 'tiempoTotalSistema', 'tiempoReposoSistema'])

dataf.to_excel(
    writer,
    index=False,
    sheet_name= 'prueba'
)

writer.save() """

C:\Users\admin\AppData\Local\Temp\ipykernel_14128\1928135098.py:14: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
